In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from pandas.tseries.offsets import DateOffset
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset

In [2]:
# Load the CSV files
df_clientes = pd.read_csv('clientes.csv')
df_transacciones = pd.read_csv('transacciones.csv')

# Merge the dataframes on the common 'id' column
merged_df = pd.merge(df_clientes, df_transacciones, on='id')

In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346011 entries, 0 to 346010
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     346011 non-null  object 
 1   fecha_nacimiento       346011 non-null  object 
 2   fecha_alta             346011 non-null  object 
 3   id_municipio           346011 non-null  int64  
 4   id_estado              346011 non-null  int64  
 5   tipo_persona           346011 non-null  object 
 6   genero                 346011 non-null  object 
 7   actividad_empresarial  346011 non-null  object 
 8   fecha                  346011 non-null  object 
 9   comercio               346011 non-null  object 
 10  giro_comercio          340423 non-null  object 
 11  tipo_venta             346011 non-null  object 
 12  monto                  346011 non-null  float64
dtypes: float64(1), int64(2), object(10)
memory usage: 34.3+ MB


In [4]:
merged_df.isnull().sum()

id                          0
fecha_nacimiento            0
fecha_alta                  0
id_municipio                0
id_estado                   0
tipo_persona                0
genero                      0
actividad_empresarial       0
fecha                       0
comercio                    0
giro_comercio            5588
tipo_venta                  0
monto                       0
dtype: int64

### Removing columns

In [5]:
cleaned_df = merged_df.drop(["id_estado", "id_municipio", "fecha_alta", "fecha_nacimiento", "actividad_empresarial", "tipo_venta", "giro_comercio", "tipo_persona", "genero"], axis=1)

In [6]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346011 entries, 0 to 346010
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        346011 non-null  object 
 1   fecha     346011 non-null  object 
 2   comercio  346011 non-null  object 
 3   monto     346011 non-null  float64
dtypes: float64(1), object(3)
memory usage: 10.6+ MB


In [7]:
cleaned_df.describe()

,monto
count,346011.000000
mean,42.290700
std,94.554414
min,1.390000
25%,9.320000
50%,17.930000
75%,39.430000
max,2297.880000


In [8]:
cleaned_df.isnull().sum()

id          0
fecha       0
comercio    0
monto       0
dtype: int64

### Transforming

In [9]:
cleaned_df['fecha'] = pd.to_datetime(cleaned_df['fecha'], format='%m/%d/%Y')

In [10]:
cleaned_df['anio'] = cleaned_df['fecha'].dt.year
cleaned_df['mes'] = cleaned_df['fecha'].dt.month
cleaned_df['dia'] = cleaned_df['fecha'].dt.day

In [11]:
cleaned_df.sort_values(by=['id', 'comercio', 'fecha'], inplace=True)
cleaned_df = cleaned_df.drop('fecha', axis=1)

In [12]:
encoder = LabelEncoder()
cleaned_df['comercio_encoded'] = encoder.fit_transform(cleaned_df['comercio'])

In [13]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 346011 entries, 332111 to 215186
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                346011 non-null  object 
 1   comercio          346011 non-null  object 
 2   monto             346011 non-null  float64
 3   anio              346011 non-null  int32  
 4   mes               346011 non-null  int32  
 5   dia               346011 non-null  int32  
 6   comercio_encoded  346011 non-null  int64  
dtypes: float64(1), int32(3), int64(1), object(2)
memory usage: 17.2+ MB


### Modelo

In [14]:
# Step 1: Create a heuristic label for subscriptions
df = cleaned_df.copy()

# Combine into a date
df['fecha'] = pd.to_datetime(dict(year=df['anio'], month=df['mes'], day=df['dia']), errors='coerce')

# Group to calculate features per (id, comercio)
features = df.groupby(['id', 'comercio']).agg({
    'fecha': ['min', 'max', 'nunique', 'count'],
    'monto': ['mean', 'std']
}).reset_index()

features.columns = ['id', 'comercio', 'fecha_min', 'fecha_max', 'fecha_nunique', 'fecha_count', 'monto_mean', 'monto_std']

# Heuristic: If they paid the same comercio ≥3 times on unique dates, it's likely a subscription
features['is_subscription'] = ((features['fecha_count'] >= 3) &
                               (features['monto_std'] < 0.5)).astype(int)

# Merge label back to main df
df = df.merge(features[['id', 'comercio', 'is_subscription']], on=['id', 'comercio'], how='left')

# Drop NA (if any) from label
df = df.dropna(subset=['is_subscription'])

# Step 2: Build a feature set
X = df[['comercio_encoded', 'anio', 'mes', 'dia', 'monto']]
y = df['is_subscription']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Step 3: Train Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     81039
           1       0.82      0.79      0.81      5464

    accuracy                           0.98     86503
   macro avg       0.90      0.89      0.90     86503
weighted avg       0.98      0.98      0.98     86503



In [15]:
# def monthly_spending(user_df):
#      # === Regression Model for Monthly Spending ===
#     monthly_spending = user_df.groupby(['anio', 'mes'])['monto'].sum().reset_index()
#     monthly_spending['month_index'] = np.arange(len(monthly_spending)) 

#     # Train regression model
#     X_train = monthly_spending[['month_index']]
#     y_train = monthly_spending['monto']
#     reg = LinearRegression()
#     reg.fit(X_train, y_train)

#     # Predict next month's total subscription spending
#     next_index = np.array([[monthly_spending['month_index'].max() + 1]])
#     predicted_total_spending = reg.predict(next_index)[0]
#     print(f"Predicted total spending for next month: {predicted_total_spending}")
#     return predicted_total_spending

In [16]:
def predict_next_month_spending(user_df):
    results = {}

    # === Total Monthly Spending Prediction ===
    monthly_spending = user_df.groupby(['anio', 'mes'])['monto'].sum().reset_index()
    monthly_spending = monthly_spending.sort_values(['anio', 'mes'])
    monthly_spending['month_index'] = np.arange(len(monthly_spending))

    X_total = monthly_spending[['month_index']]
    y_total = monthly_spending['monto']
    reg_total = LinearRegression()
    reg_total.fit(X_total, y_total)

    next_month_index = np.array([[monthly_spending['month_index'].max() + 1]])
    predicted_total = reg_total.predict(next_month_index)[0]
    results['total'] = predicted_total

    print(f"📈 Predicted total spending next month: ${predicted_total:.2f}")

    # === Per-Merchant Monthly Spending Prediction ===
    merchants = user_df['comercio'].unique()
    results['per_merchant'] = {}

    for merchant in merchants:
        merchant_df = user_df[user_df['comercio'] == merchant]
        monthly = merchant_df.groupby(['anio', 'mes'])['monto'].sum().reset_index()
        monthly = monthly.sort_values(['anio', 'mes'])
        monthly['month_index'] = np.arange(len(monthly))

        if len(monthly) < 2:
            # Not enough data points for regression
            results['per_merchant'][merchant] = monthly['monto'].iloc[-1]
            continue

        X = monthly[['month_index']]
        y = monthly['monto']
        reg = LinearRegression()
        reg.fit(X, y)

        next_idx = np.array([[monthly['month_index'].max() + 1]])
        pred = reg.predict(next_idx)[0]
        if pred < 0:
            continue
        results['per_merchant'][merchant] = pred

        print(f"🔸 {merchant}: Predicted spending next month: ${pred:.2f}")

    return predicted_total, results

In [17]:
def iconic_expense(user_df):
    unique_counts = user_df['comercio'].value_counts()
    most_unique_commerce = unique_counts.idxmin()
    most_unique_count = unique_counts.min()

    return most_unique_commerce, most_unique_count

In [ ]:
def predict_next_month_subscriptions(user_id):
    # Filter transactions for the user
    user_df = df[df['id'] == user_id].copy()
    if user_df.empty:
        print("No transactions found for user:", user_id)
        return pd.DataFrame(), 0.0

    # Predict current subscriptions
    feature_columns = ['comercio_encoded', 'anio', 'mes', 'dia', 'monto']
    user_df['subscription_prediction'] = model.predict(user_df[feature_columns])
    predicted_subs = user_df[user_df['subscription_prediction'] == 1].copy()
    if predicted_subs.empty:
        print("No subscriptions predicted for this user")
        return pd.DataFrame(), 0.0

    # Get most recent date
    most_recent_date = user_df['fecha'].max()
    
    # Cutoff date (3 months back)
    cutoff_date = most_recent_date - DateOffset(months=3)

    # Filter to only subscriptions with activity in the last 3 months
    recent_subs = predicted_subs[predicted_subs['fecha'] >= cutoff_date].copy()
    if recent_subs.empty:
        print("No recent subscriptions found for this user")
        return pd.DataFrame(), 0.0

    # Get next month and year
    last_month = most_recent_date.month
    next_month = last_month + 1 if last_month < 12 else 1
    next_year = most_recent_date.year if next_month != 1 else most_recent_date.year + 1

    # --- Prepare next month subscription candidates with all frequent days ---
    # Get all frequent days (modes) per comercio
    frequent_days = recent_subs.groupby('comercio')['dia'].agg(lambda x: x.mode().tolist()).reset_index()

    # Explode to have one row per frequent day
    frequent_days = frequent_days.explode('dia')

    # Base features (drop duplicates)
    base = recent_subs.drop_duplicates(subset='comercio', keep='first').drop(columns='dia')

    # Merge back to get a row per (comercio, frequent day)
    next_month_candidates = frequent_days.merge(base, on='comercio')
    next_month_candidates['mes'] = next_month
    next_month_candidates['anio'] = next_year

    # Predict subscriptions for next month
    X_next_month = next_month_candidates[feature_columns]
    next_month_candidates['subscription_prediction'] = model.predict(X_next_month)
    next_month_subs = next_month_candidates[next_month_candidates['subscription_prediction'] == 1]
   
    return next_month_subs[['comercio', 'monto', 'anio', 'mes', 'dia']]

In [19]:
def all_predictions(user_id):
    user_df = df[df['id'] == user_id].copy()
    if user_df.empty:
        print("No transactions found for user:", user_id)
        return None

    # Predict next month spending
    total_spending, per_merchant_spending = predict_next_month_spending(user_df)

    # Predict subscriptions
    predicted_subs = predict_next_month_subscriptions(user_id)

    # Iconic expense
    iconic_commerce, iconic_count = iconic_expense(user_df)

    return {
        'total_spending': total_spending,
        'per_merchant_spending': per_merchant_spending,
        'predicted_subs': predicted_subs,
        'iconic_commerce': iconic_commerce,
        'iconic_count': iconic_count
    }

In [20]:
user_id = '01d12d1c5b58348995b71cf1d5eb9257fa64d95b'  # Replace with a real user ID
user_df = df[df['id'] == user_id]

all_predictions_result = all_predictions(user_id)
if all_predictions_result:
    print("Total Spending Next Month:", all_predictions_result['total_spending'])
    print("Per Merchant Spending Next Month:", all_predictions_result['per_merchant_spending'])
    print("Predicted Subscriptions Next Month:\n", all_predictions_result['predicted_subs'])
    print("Iconic Commerce:", all_predictions_result['iconic_commerce'])
    print("Iconic Count:", all_predictions_result['iconic_count'])

📈 Predicted total spending next month: $1931.46
🔸 ALIEXPRESS: Predicted spending next month: $556.79
🔸 AMAZON: Predicted spending next month: $134.85
🔸 AMAZON PRIME: Predicted spending next month: $21.45
🔸 CRUNCHYROLL: Predicted spending next month: $17.36
🔸 DIDI: Predicted spending next month: $24.21
🔸 DIDIFOOD: Predicted spending next month: $30.48
🔸 MELIMAS: Predicted spending next month: $18.58
🔸 MERCADO PAGO: Predicted spending next month: $1095.74
🔸 MERCADOPAGO: Predicted spending next month: $594.59
🔸 NETFLIX: Predicted spending next month: $36.58
🔸 RAPPI: Predicted spending next month: $92.63
🔸 RAPPIPRO: Predicted spending next month: $12.76
🔸 TOTALPLAY: Predicted spending next month: $101.33
🔸 UNDOSTRES: Predicted spending next month: $104.06
Cutoff date: 2022-10-30 00:00:00
Most recent date: 2023-01-30 00:00:00
Next month: 2, Next year: 2023
Total Spending Next Month: 1931.463076923077
Per Merchant Spending Next Month: {'total': np.float64(1931.463076923077), 'per_merchant': 

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3

In [21]:
predict_next_month_subscriptions(user_id)

Cutoff date: 2022-10-30 00:00:00
Most recent date: 2023-01-30 00:00:00
Next month: 2, Next year: 2023


,comercio,monto,anio,mes,dia
0,AMAZON PRIME,11.62,2023,2,2
1,CRUNCHYROLL,17.36,2023,2,26
